> *Premature optimization is the root of all evil.*
> <div align="right"> Donald Knuth </div>

In this post, we'll try to optimize the execution time of  
the sliding-window burst search algorithm 
([Fries 1998](http://dx.doi.org/10.1021/jp980965t)) implemented in python. 
I will use several python optimization techniques, 
namely vectorization (numpy), "loop unwrapping" and compilation to machine code 
with Cython and Numba.

As of why I choose python, you can read the famous Jake Vanderplas' post: 
[Why Python is the Last Language You'll Have To Learn](https://jakevdp.github.io/blog/2012/09/20/why-python-is-the-last/). 
Or, as John Cook synthetically put it:

> I’d rather do mathematics in a general programming language than do general programming in a mathematical language.

(from John's recent post 
[Scientific computing in Python](http://www.johndcook.com/blog/2015/07/16/scientific-computing-in-python/)).

# Burst search

To give some context, in the analysis of freely-diffusing single-molecule experiments, 
the burst search algorithm is the central step that allows identifying 
bursts of photon emitted by single molecules during their diffusion
through an excitation volume.
Here we use a simplified burst search that saves only start and stop times
of each burst. For a complete implementation see the 
[FRETBursts](http://tritemio.github.io/FRETBursts/) burst analysis 
software ([code here](https://github.com/tritemio/FRETBursts/tree/master/fretbursts/burstsearch)).

Briefly, a burst search algorithm tries to identifies bursts in a long stream 
of events. In single-molecule fluorescence experiments, this stream is
represented by photon arrival times (timestamps) with a resolution of a few 
nanoseconds.

A common algorithm, introduced by the Seidel group
([Fries 1998](http://dx.doi.org/10.1021/jp980965t)), consists in using a 
sliding windows of duration $T$ and identifying bursts when at least $m$ photons 
lie in this window.
The final step of selecting bursts by size (counts, or number of photons)
is computationally inexpensive and it will be ignored in this post.

Numerically, we need to "slide" the window in discrete steps, and since 
the photon arrival times are stochastic, it makes sense to place
the windows start in correspondence with each timestamp $t_i$ and check 
if there are at least $m$ photons between $t_i$ and $t_i + T$.

But how can we "check if there are at least $m$ photons between 
$t_i$ and $t_i + T$"? In principle, we can count how many photons 
lie in the current time windows and start the burst if there are at least $m$. 
This would require, for each timestamp $t_i$, looping through a variable
number of timestamps ($n$), until $t_{i+n} - t_i >=T$ then check if 
$n >= m$. In total, we would loop through the full `timestamps` array many times.

Equivalently, we can loop over consecutive m-tuple of photons, and 
test if the time difference between the last and the first timestamp 
is $\le T$. This latter approach is much easier to implement and more
efficient as it requires looping through the `timestamps` exactly once.

For the sake of this post, we assume that each burst is characterized 
by only a start and stop time. Finally, since the number of bursts 
is not known in advance, we'll use a list to collect the bursts
found during the search.

# Prepare the data

To test different burst search implementation we can use a single-molecule FRET dataset [available on figshare](http://figshare.com/articles/Example_smFRET_data_files_in_Photon_HDF5_format/1456362). The file are in [Photon-HDF5](http://photon-hdf5.org), so we can load its content with a HDF5 library, such as [pytables](http://www.pytables.org/).

For this post we only need to load the `timestamps` array, which is here converted in seconds:

In [1]:
import tables
import numpy as np

In [2]:
filename = "data/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5"

In [3]:
with tables.open_file(filename) as h5file:
    timestamps = h5file.root.photon_data.timestamps.read()
    timestamps = timestamps*h5file.root.photon_data.timestamps_specs.timestamps_unit.read()

In [4]:
timestamps

array([  1.83558750e-03,   2.35056250e-03,   3.67655000e-03, ...,
         5.99998296e+02,   5.99998472e+02,   5.99999442e+02])

In [5]:
timestamps.size

2683962

# Implementation 1: for-loop

The previously described algorithm can be expressed quite naturally with a for-loop:

In [6]:
def burst_search(times, m, T):
    in_burst = False
    bursts = []
    for i in range(len(times) - m - 1):
        if times[i + m - 1] - times[i] <= T:
            if not in_burst:
                in_burst = True
                istart = i
        elif in_burst:
            in_burst = False
            bursts.append((times[istart], times[i+m-1]))
    return bursts

The code is straightforward to read:

1. the $i$ variable loops over the timestamps index
2. if the $m$ consecutive photons starting at $t_i$ are within a window $\le T$
  1. if a burst is not already started, start the burst and save the start time
3. Otherwise, if we are inside a burst, stop the burst and save the stop time

Let's run it:

In [7]:
bursts_py = burst_search(timestamps, 10, 100e-6)

In [8]:
len(bursts_py)

18529

In [9]:
%timeit burst_search(timestamps, 10, 100e-6)

1 loops, best of 3: 1.04 s per loop


So, we found 18529 bursts and the execution took around a second. 
It may seem we got the results "fast enough". However, 
when dealing with larger files (longer measurement, multi-spot, etc...) 
or when we need to interactively explore the effect of burst search 
parameters on burst populations we need a faster burst search.

Next sections show various optimization approaches.

# Finding the bottlenecks

First step of optimization is always identifying the 
[bottlenecks](https://en.wikipedia.org/wiki/Bottleneck_(software). 

To measure of the execution time line-by-line we can use 
`line_profiler` (install it through PIP or conda).

For a more detailed overview of different profiling techniques
see the excellent Cyrille Rossant's post 
[Profiling and optimizing Python code](http://cyrille.rossant.net/profiling-and-optimizing-python-code/).

Here we load the profiler and run out function through it:

In [10]:
%load_ext line_profiler

In [11]:
%lprun -f burst_search burst_search(timestamps, 10, 100e-6)

```
Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def burst_search(times, m, T):
     2         1            2      2.0      0.0      in_burst = False
     3         1            1      1.0      0.0      bursts = []
     4   2683952      1195136      0.4     25.9      for i in range(len(times) - m - 1):
     5   2683951      2246412      0.8     48.6          if times[i + m - 1] - times[i] <= T:
     6    232747       103669      0.4      2.2              if not in_burst:
     7     18529         7924      0.4      0.2                  in_burst = True
     8     18529         8029      0.4      0.2                  istart = i
     9   2451204      1036893      0.4     22.4          elif in_burst:
    10     18529         8080      0.4      0.2              in_burst = False
    11     18529        16394      0.9      0.4              bursts.append((times[istart], times[i+m-1]))
    12         1            1      1.0      0.0      return bursts
```

The most unespected result, for me, was to find that the simple branching 
(e.g. line 9) accounts for a significant 20% of execution time.
Except for that, we see that looping over >2 millions of elements and 
computing `times[i+m] - times[i] <= T` (line 5) at each cycle is where
the function spends the most time. The list-appending, conversely,
adds a negligible contribution since it is performed once per burst
(not at each cycle). 

Numpy array are notoriously slow when accessed element by element.
Therefore a big portion of line 5 execution time may be due to numpy arrary indexing.
We can prove this by expanding line 5 in multiple lines to separate the element 
access from comparison and branching:

In [12]:
def burst_search_profile(times, m, T):
    in_burst = False
    bursts = []
    for i in range(len(times) - m - 1):
        t2 = times[i + m - 1]
        t1 = times[i]
        rate_above_threshold = t2 - t1 <= T
        if rate_above_threshold:
            if not in_burst:
                in_burst = True
                istart = i
        elif in_burst:
            in_burst = False
            bursts.append((times[istart], times[i+m-1]))
    return bursts

In [13]:
%lprun -f burst_search_profile burst_search_profile(timestamps, 10, 100e-6)

```
Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def burst_search_profile(times, m, T):
     2         1           20     20.0      0.0      in_burst = False
     3         1            1      1.0      0.0      bursts = []
     4   2683952      1303409      0.5     14.8      for i in range(len(times) - m - 1):
     5   2683951      1806225      0.7     20.5          t2 = times[i + m - 1]
     6   2683951      1526900      0.6     17.3          t1 = times[i]
     7   2683951      1607583      0.6     18.2          rate_above_threshold = t2 - t1 <= T
     8   2683951      1284202      0.5     14.5          if rate_above_threshold:
     9    232747       111491      0.5      1.3              if not in_burst:
    10     18529         8883      0.5      0.1                  in_burst = True
    11     18529         8949      0.5      0.1                  istart = i
    12   2451204      1142937      0.5     12.9          elif in_burst:
    13     18529         8813      0.5      0.1              in_burst = False
    14     18529        20464      1.1      0.2              bursts.append((times[istart], times[i+m-1]))
    15         1            4      4.0      0.0      return bursts
```

We see that the array element access (lines 5 and 6) accounts for almost 40%
of the total execution time. 

A workaround to reduce array element access time is using a `memoryview`
(a [built-in python object](https://docs.python.org/3.5/library/stdtypes.html#memory-views))
to access the data in the numpy array.
Since the `memoryview` shares the data with the original numpy array
we avoid wasting RAM and waiting for memory copying. 
Let's call the function `burst_search` passing a memoryview instead 
of a numpy array:

In [14]:
%timeit burst_search(memoryview(timestamps), 10, 100e-6)

1 loops, best of 3: 637 ms per loop


This little trick provides almost 40% speed increase,
rendering the element access time negligible. Note, however, that is not
always possible to use `memoryview` because it does not support
all the advanced numpy indexing and array operations.

# Implementation 2: for-loop compiled


The `burst_search()` function has a "hot" for-loop containing 
an if-else branching that depends on array elements computations. 
In this case, it is hard to devise an implementation for the classical technique 
of replacing the for-loop with vectorization (i.e. arrays operations). 
Therefore, the straighforward optimization path is compiling the function 
to machine code. This can be achieved by using [cython](http://cython.org/).

Cython extends the python syntax, and allows to statically translate 
python to C (that, eventually, needs to be compiled to machine code).  
To allows the compiler to optimize the function, we need to specify 
the types of the variables used inside the loop.

The cython version of the previous algorithm is:

In [15]:
%load_ext Cython

In [16]:
%%cython
cimport numpy as np

def burst_search_cy(np.float64_t[:] times, np.int16_t m, np.float64_t T):
    cdef int i
    cdef np.int64_t istart
    cdef np.uint8_t in_bursts
    
    in_burst = 0
    bursts = []
    for i in range(times.size - m - 1):
        if times[i + m - 1] - times[i] <= T:
            if not in_burst:
                in_burst = 1
                istart = i
        elif in_burst:
            in_burst = 0
            bursts.append((times[istart], times[i+m-1]))
    return bursts

In the Jupyter notebook, we use the `%%cython` 
[magic command](https://ipython.org/ipython-doc/dev/interactive/tutorial.html) 
(included in the `cython` package) to easily compile the function.
Outside the notebook, we can setup `distutils` (or `setuptools`) to handle the
compilation step (see [cython documentation](http://docs.cython.org/)).

Apart from the boiler-plate import, I added type definitions to 
the function arguments. The cython types have the same name as 
numpy types with and additional `_t`. For the first argument
which is an array we use the syntax `np.float64_t[:] times`
that defines a [Cython Memoryview](http://docs.cython.org/src/userguide/memoryviews.html)
(like a python memoryview but faster).
To read more about memoryviews see this post from Jake Vanderplas: [Memoryview Benchmarks](https://jakevdp.github.io/blog/2012/08/08/memoryview-benchmarks/).

Let's run our cython function:

In [17]:
bursts_cy = burst_search_cy(timestamps, 10, 100e-6)

In [18]:
assert bursts_cy == bursts_py

In [19]:
%timeit burst_search_cy(timestamps, 10, 100e-6)

100 loops, best of 3: 8.72 ms per loop


Note that the execution time dropped to less than 10ms, a whooping 
100x speed increase compared to the pure python version. And all 
we have done is adding a few variable declarations!

Another optimization tool emerged in the last couple of years is 
[Numba](http://numba.pydata.org/). 
Numba is a Just-in-Time (JIT) compiler which analyze code 
during execution and translates it to machine code on-fly. Under 
the hood it uses the [LLVM compiler](https://en.wikipedia.org/wiki/LLVM) 
to translate python function to machine code.

In principle, numba can perform more advanced optimizations than 
cython and there are, in some cases, reports of 2x speed improvements 
vs cython.

Numba is even easier than cython to use: we just need to add a single 
line to decorate the fuction (`@numba.jit`):

In [20]:
import numba

In [21]:
@numba.jit
def burst_search_numba(times, m, T):
    in_burst = False
    bursts = []
    istart = 0
    for i in range(times.size - m - 1):
        if times[i + m - 1] - times[i] <= T:
            if not in_burst:
                in_burst = True
                istart = i
        elif in_burst:
            in_burst = False
            bursts.append((times[istart], times[i+m-1]))
    return bursts

Differently from cython, numba code is just python code with no special syntax.

Let's check the execution time:

In [22]:
bursts_numba = burst_search_numba(timestamps, 10, 100e-6)

In [23]:
assert bursts_numba == bursts_py

In [24]:
%timeit burst_search_numba(timestamps, 10, 100e-6)

100 loops, best of 3: 8.24 ms per loop


Like the cython version, numba version runs in less than 10 ms.

You may have noticed an additional line (`istart = 0`) which
I added to helps Numba to infer the type (knowing that `istart` 
is an int64 numba can do more aggressive optimizations). 

There are still some corner cases which Numba cannot
optimize, but these cases are becoming fewer at each release. 
For example in the previous version 0.20, Numba was not able to optimize 
the present example (which was running at pure-python speed). 
With current Numba version (0.22), instead, we reaches pure-C speed
probably thanks to the [new list optimizations](http://numba.pydata.org/numba-doc/0.22.1/release-notes.html).

# Implementation 3: vectorized numpy

We still optimize the pure python + numpy function. We will not 
completely remove the for-loop but we will move some heavy 
computation out of it.

As shown before, the bottleneck is the operation 
`t[i+m] - t[i] <= T`. In particular the array element access 
is quite slow.

At cost of higher memory usage we can perform the difference 
outside the loop:

In [25]:
def burst_search_numpy1(times, m, T):
    in_burst = False
    bursts = []
    delta_t = (times[m-1:] - times[:times.size-m+1])
    for i in range(times.size-m+1):
        if delta_t[i] <= T:
            if not in_burst:
                in_burst = True
                start = times[i]
        elif in_burst:
            in_burst = False
            bursts.append((start, times[i+m-1]))
    return bursts

In [26]:
bursts_numpy1 = burst_search_numpy1(timestamps, 10, 100e-6)
assert bursts_numpy1 == bursts_py
%timeit burst_search_numpy1(timestamps, 10, 100e-6)

1 loops, best of 3: 465 ms per loop


we got a 2x speed improvement, not bad. 

Thinking further on this lines, we can also move the comparison 
outside the loop:

In [27]:
def burst_search_numpy2(times, m, T):
    in_burst = False
    bursts = []
    above_min_rate = (times[m-1:] - times[:times.size-m+1]) <= T
    for i in range(len(times)-m+1):
        if above_min_rate[i]:
            if not in_burst:
                in_burst = True
                start = times[i]
        elif in_burst:
            in_burst = False
            bursts.append((start, times[i+m-1]))
    return bursts

In [28]:
bursts_numpy2 = burst_search_numpy2(timestamps, 10, 100e-6)
assert bursts_numpy2 == bursts_py
%timeit burst_search_numpy2(timestamps, 10, 100e-6)

1 loops, best of 3: 271 ms per loop


The line profiler shows that the simple item access (line 6) takes 30% of the time.
This is not surprising because numpy has some overhead when accessing a single
element because of all the fancy indexing it supports. Since here we are simply
iterating over all the elements we can use the iterator to get the array elements.


In [29]:
def burst_search_numpy3(times, m, T):
    in_burst = False
    bursts = []
    
    max_index = times.size-m+1
    above_min_rate = ((times[m-1:] - times[:max_index]) <= T)
    
    for i, above_min_rate_ in enumerate(above_min_rate):
        if above_min_rate_:
            if not in_burst:
                in_burst = True
                start = times[i]
        elif in_burst:
            in_burst = False
            bursts.append((start, times[i+m-1]))
    return bursts

In [30]:
bursts_numpy3 = burst_search_numpy3(timestamps, 10, 100e-6)
assert bursts_numpy3 == bursts_py
%timeit burst_search_numpy3(timestamps, 10, 100e-6)

1 loops, best of 3: 244 ms per loop


Finally, we can optimize line 13, the else branch that is executed at almost every cycle.
To avoid this test we can slightly unwrap the loop:

- first we `continue` if not above the rate.
- when "not continuing", we are inside a burst and we do a mini internal loop until the burst is over
- the main loop resumes from a position updated by the inner loop

The trick is using the same iterator in the two nested loops:

In [31]:
def burst_search_numpy4(times, m, T):
    bursts = []
    
    max_index = times.size-m+1
    below_min_rate = memoryview((times[m-1:] - times[:max_index]) > T)
    
    iter_i_belowrate = enumerate((below_min_rate))
    for i, below_min_rate_ in iter_i_belowrate:
        if below_min_rate_: continue           
        
        start = times[i]
        for i, below_min_rate_ in iter_i_belowrate:
            if below_min_rate_: break
        
        bursts.append((start, times[i+m-1]))
    return bursts

In [32]:
bursts_numpy4 = burst_search_numpy4(timestamps, 10, 100e-6)
assert bursts_numpy4 == bursts_py
%timeit burst_search_numpy4(timestamps, 10, 100e-6)

10 loops, best of 3: 197 ms per loop


Note that I used `memoryview`for a slightly faster iteration than with a numpy array.

Finally we use a simpler iterator only for the index and access `below_min_rate`
element by element:

In [33]:
def burst_search_numpy5(times, m, T):
    bursts = []
    below_min_rate = memoryview((times[m-1:] - times[:times.size-m+1]) > T)
    
    iter_i = iter(range(len(times)-m+1))
    for i in iter_i:
        if below_min_rate[i]:
            continue
        
        start = times[i]
        for i in iter_i:
            if below_min_rate[i]:
                break
        
        bursts.append((start, times[i+m-1]))
    return bursts

In [34]:
bursts_numpy5 = burst_search_numpy5(timestamps, 10, 100e-6)
assert bursts_numpy5 == bursts_py
%timeit burst_search_numpy5(timestamps, 10, 100e-6)

10 loops, best of 3: 197 ms per loop


Overall the last optimizations (3-5) cut another 25-30% to the execution time.
More importantly, I found the last version (5) the easiest to read, 
mainly because of the elimination of the state variable `in_burst`
and use a simpler iterator.

# Conclusion

Starting from a direct pure-python algorithm implementation we found
how to profile it (using `line_profiler`) and optimize it in Cython 
and Numba obtaining, in both cases, a 100x (100 fold!) speed-up.

Furthermore, even if the burst search algorithm is not easy to vectorize, 
a careful analysis of the bottlenecks allowed 
to obtain a 5x speed-up using only python + numpy.

With all these easy-to-use optimization options offered by python, 
it is easy to fall in the trap of premature optimization. To avoid it,
always perform optimization as the last step, and focus only on the 
bottlenecks highlighted by a profiler.
When in doubt, prefer clarity over speed.

# See also

- [FRETBursts](http://tritemio.github.io/FRETBursts/) A burst analysis 
  software implementing the techniques illustrated in this post 
  in a more complete real-world scenario.
  
> This post is written in Jupyter Notebook. The original notebook
> can be downloaded [here](https://github.com/tritemio/smbits/blob/master/content/2015-09/optimize-burst-search-python.ipynb). The content released as [CC BY 2.0](https://creativecommons.org/licenses/by/2.0/).